In [74]:
# Import dependencies

import json
import pandas as pd
import re

In [75]:
# Loads the json into an object for the DataFrame.  The JSON must be in the same folder.

with open("neilsendma.json","r") as read_file:
    dmaJSON = json.load(read_file)

In [76]:
# Reads each JSON entry into a list, and then builds dmaDF DataFrame from that list.
# A final row is added to account for unnamed markets.  It is given the DMA ID of '999' and
# it's name is set to 'Null'.  Also, the unique DMA ID is defined as the index.

marketList = []
dmaDF = pd.DataFrame

for key, value in dmaJSON.items():
    marketList.append([key, value['Designated Market Area (DMA)'], []])
dmaDF = pd.DataFrame(marketList,columns = ['dmaID','marketName','aliasList'])
dmaDF = dmaDF.append(pd.Series(['999','Null',[]], index = dmaDF.columns), ignore_index=True)
dmaDF.set_index('dmaID',inplace=True)

In [77]:
dmaDF.head()

,marketName,aliasList
dmaID,,
500,Portland-Auburn,[]
501,New York,[]
502,Binghamton,[]
503,Macon,[]
504,Philadelphia,[]


In [78]:
dmaDF.tail()

,marketName,aliasList
dmaID,,
862,Sacramnto-Stkton-Modesto,[]
866,Fresno-Visalia,[]
868,Chico-Redding,[]
881,Spokane,[]
999,Null,[]


In [79]:
# Regular expressions used for adding DMA region aliases to the master DataFrame (dmaDF).  Since each data source can have a 
# unique way of structuring it's city-DMA data, a new regex may need to be written for each new sheet.  

def RegexSplitDMA(inString): # Returns a list of ['City Name', 'DMA#']
    try:
        prog = re.compile(pattern = r"([\w\W]*)\s\(([0-9]{3})\)")
        return([prog.match(inString).group(1).strip(), prog.match(inString).group(2)])
    except:
        return ['None','999']
    
def RegexSplitInfluencer(inString): # Will be used on future datasets.  Returns just the City Name
    prog = re.compile(pattern = r"[\w\W]*\s-\s([\w|\s]+)")
    return(prog.match(inString).group(1))

In [80]:
# Reads data from the 'KRG Influencer Test' csv in order to collect aliases for cities.
# Only one column is needed since it contains 'City name (DMA#)'.  The column is split by RegexSplitDMA().
# The DMA ID returned from the split is checked against master DataFrame, and if the city name is not 
# included in 'aliasList' for that DMA, then it is added.

tempDF = pd.DataFrame
elementList = []

tempDF = pd.read_csv('KRG Influencer Test Impressions by DMA.csv', header = 0)
tempDF.columns = ['date','campaign','dma','impressions']
for element in tempDF.dma:
    elementList = RegexSplitDMA(element)
    if elementList[0] not in dmaDF.loc[elementList[1],'aliasList']:
        dmaDF.loc[elementList[1],'aliasList'].append(elementList[0])

In [81]:
dmaDF.head()

,marketName,aliasList
dmaID,,
500,Portland-Auburn,[portland-auburn]
501,New York,[new york]
502,Binghamton,[binghamton]
503,Macon,[macon]
504,Philadelphia,[philadelphia]


In [82]:
dmaDF.tail()

,marketName,aliasList
dmaID,,
862,Sacramnto-Stkton-Modesto,[sacramnto-stkton-modesto]
866,Fresno-Visalia,[fresno-visalia]
868,Chico-Redding,[chico-redding]
881,Spokane,[spokane]
999,Null,[None]


In [83]:
# Another csv is read in order to build out more aliases for each region.  This uses a csv with
# hand-tagged cities which do not match with DMA region names.  These are usually cities which fall within the DMA region
# and hand to be individually looked-up to match with a DMA.  Once DMA is determined, and if the city name is not already
# part of 'aliasList', then it is added to the master DataFrame

tempDF = pd.DataFrame
tempDF = pd.read_csv('6626 alt cities.csv', header = None)
tempDF.columns = ['type','city','dma','unknownValue']

for row in tempDF.iterrows():
    if tempDF.loc[row[0],'city'] not in dmaDF.loc[str(tempDF.loc[row[0],str('dma')]),'aliasList']:
        dmaDF.loc[str(tempDF.loc[row[0],str('dma')]),'aliasList'].append(tempDF.loc[row[0],'city'])

In [84]:
dmaDF.head()

,marketName,aliasList
dmaID,,
500,Portland-Auburn,[portland-auburn]
501,New York,"[new york, Norwalk, CT, New York, Danbury, CT,..."
502,Binghamton,[binghamton]
503,Macon,[macon]
504,Philadelphia,"[philadelphia, Philadelphia]"


In [85]:
dmaDF.tail()

,marketName,aliasList
dmaID,,
862,Sacramnto-Stkton-Modesto,[sacramnto-stkton-modesto]
866,Fresno-Visalia,[fresno-visalia]
868,Chico-Redding,"[chico-redding, Redding, CA]"
881,Spokane,[spokane]
999,Null,[None]
